Lo script seguente itera in ordine alfabetico i file inseriti nella cartella "files" e, a seconda del tipo, li sposta nella relativa sottocartella. Se la sottocartella non esiste lo script la crea automaticamente. Viene anche creato un file "recap.csv" in cui ogni volta che viene spostato un file viene aggiornato con il relativo nome, tipo e dimensione.

In [6]:
#Mi sposto fisicamente sulla cartella 'files'

import os
os.chdir(os.getcwd()+'\\fileorganizer\\files')

In [15]:
#Creo una lista dei file presenti nella cartella files

files = os.listdir(os.getcwd())

#Definisco le variabili del percorso di partenza dei file da spostare e quelle dei percorsi delle future sottocartelle

source = os.getcwd()
dest_images =source+'\\images'
dest_docs = source+ '\\docs'
dest_audio = source+ '\\audio'


#Creo delle liste dei formati suddivisi per tipo. Non creo la lista "audio" visto che è presente un solo formato "audio"

image = [".jpg",".jpeg",".png"]
doc = [".odt",".txt"]
audio = [".mp3"]

#Creo file recap.csv con '"name, "type", "size(B)"' come "header" di colonne

import shutil
import csv
with open('recap.csv', 'a',newline='') as recap: 
    writer = csv.writer(recap) 
    writer.writerow(["name", "type", "size(B)"])
    
#Iterando sulla cartella files creo le variabili che mi permettono di stampare le informazioni e di completare il file recap
#Confronto l'estensione di ogni file con le liste precedentemente create per ottenere il "tipo"
#In base all'estensione del file sposto il file dalla cartella di partenza alla sua sottocartella tramite "shutil.move"
#Se la sottocartella non esiste viene creata tramite "os.makedirs"

    for file in sorted(files): 
        nome,est= os.path.splitext(file)
        size = os.path.getsize(file)
        tipo = ""
        if est in image:
            tipo = "image"
            print(nome,"type:{} size:{}".format(tipo,size))
        elif est in doc:
            tipo = "doc"
            print(nome,"type:{} size:{}".format(tipo,size))
        elif est in audio:
            tipo = "audio"
            print(nome,"type:{} size:{}".format(tipo,size))
        writer.writerows([[name,tipo,size]]) 

        if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpeg"):
            if os.path.exists(dest_images):
                shutil.move(source + "\\" + file, dest_images)
            else:
                os.makedirs(dest_images)
                shutil.move(source + "\\" + file,dest_images)
    
        if file.endswith(".txt") or file.endswith(".odt"):
            if os.path.exists(dest_docs):
                shutil.move(source + "\\" + file, dest_docs)
            else:
                os.makedirs(dest_docs)
                shutil.move(source +  "\\" + file,dest_docs)
            
        if file.endswith(".mp3"):
            if os.path.exists(dest_audio):
                shutil.move(source + "\\" + file, dest_audio)
            else:
                os.makedirs(dest_audio)
                shutil.move(source +  "\\" + file,dest_audio)

bw type:image size:94926
ciao type:doc size:12
daffodil type:image size:24657
eclipse type:image size:64243
pippo type:doc size:8299
song1 type:audio size:1087849
song2 type:audio size:764176
trump type:image size:10195


Il seguente script itera invece sulla cartella immagini e costruisce una tabella riassuntiva contentente le informazioni di colore dei file.

In [16]:
#Itero sulla cartella "images" usando "glob.glob" che mi restituisce tutti i file presenti sulla cartella(path compreso)
#Nello stesso ciclo for apro ogni file tramite "Image.open" e li trasformo in array
#Ottengo il nome di ogni file che inserirò nelle informazioni richieste da stampare

import glob
from PIL import Image
import numpy as np

list_info = []
for filename in glob.glob(os.getcwd()+'\\images\\*'): 
    im=Image.open(filename)
    im_array = np.array(im)
    filenames = os.path.basename(filename)
    nome = os.path.splitext(filenames)[0] 
    im.close()

#Creo un dizionario che ha come chiavi tutti i dati che servono da stampare
#Tramite "im_array.shape" ottengo altezza e larghezza di ogni immagine
#Se l'array è bidimensionale faccio la media di tutti i valori dell'array e ottengo il valore della scala di grigi
#Se l'array è tridimensionale faccio la media di tutti i valori dell'array unendoli in una tupla e ottengo i valori 'R,G,B'
#Se l'array è tridimensionale ed ha 4 livelli di colore ottengo anche il canale 'ALPHA'
#Aggiungo tutti i dati ad una lista precedentemente creata

    
    im_info = {'name': nome,'height':0,'width':0,'grayscale':0,'R':0,'G':0,'B':0,'ALPHA':0}
    im_info['height'] = im_array.shape[0]
    im_info['width']=im_array.shape[1]
    if len(im_array.shape) == 2:
        im_info['grayscale'] = np.mean(im_array)
    elif len(im_array.shape) >= 3:
        im_info['R'] = np.mean(im_array,axis = (0,1))[0]
        im_info['G'] = np.mean(im_array,axis = (0,1))[1]
        im_info['B'] = np.mean(im_array,axis = (0,1))[2]
        if im_array.shape[2] == 4:
            im_info['ALPHA'] = np.mean(im_array,axis = (0,1))[3]
        
    list_info.append(im_info)

#Creo una tabella tramite la libreria "tabulate" inserendo la lista di dizionari creata precedentemente
#La tabella avrà come colonne le chiavi dei dizionari

from tabulate import tabulate    

print(tabulate(list_info, floatfmt=".2f",tablefmt='fancy_grid',headers = "keys"))

╒══════════╤══════════╤═════════╤═════════════╤════════╤════════╤═══════╤═════════╕
│ name     │   height │   width │   grayscale │      R │      G │     B │   ALPHA │
╞══════════╪══════════╪═════════╪═════════════╪════════╪════════╪═══════╪═════════╡
│ bw       │      512 │     512 │       21.48 │   0.00 │   0.00 │  0.00 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ daffodil │      500 │     335 │        0.00 │ 109.25 │  85.56 │  4.97 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ eclipse  │      256 │     256 │        0.00 │ 109.05 │ 109.52 │ 39.85 │  133.59 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ trump    │      183 │     275 │        0.00 │  97.01 │  98.99 │ 90.92 │    0.00 │
╘══════════╧══════════╧═════════╧═════════════╧════════╧════════╧═══════╧═════════╛
